<a href="https://colab.research.google.com/github/chijete/sherpa-onnx-wass/blob/main/sherpa-onnx/sherpa_onnx_wasm_asr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WebAssembly for ASR (sherpa-onnx)

This colab notebook shows you how to run
https://k2-fsa.github.io/sherpa/onnx/wasm/index.html

## Install Emscripten

In [ ]:
%%shell

git clone https://github.com/emscripten-core/emsdk.git
cd emsdk
git pull
./emsdk install 3.1.48
./emsdk activate 3.1.48

# Specific version: https://github.com/k2-fsa/sherpa-onnx/issues/2365

source ./emsdk_env.sh

emcc -v

Cloning into 'emsdk'...
remote: Enumerating objects: 4512, done.
remote: Counting objects: 100% (665/665), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 4512 (delta 614), reused 572 (delta 572), pack-reused 3847 (from 4)
Receiving objects: 100% (4512/4512), 2.57 MiB | 10.65 MiB/s, done.
Resolving deltas: 100% (2969/2969), done.
Already up to date.
Resolving SDK alias 'latest' to '4.0.12'
Resolving SDK version '4.0.12' to 'sdk-releases-209b886304498eff50dd835850dc5715803401ed-64bit'
Installing SDK 'sdk-releases-209b886304498eff50dd835850dc5715803401ed-64bit'..
Installing tool 'node-22.16.0-64bit'..
Downloading: /content/emsdk/downloads/node-v22.16.0-linux-x64.tar.xz from https://storage.googleapis.com/webassembly/emscripten-releases-builds/deps/node-v22.16.0-linux-x64.tar.xz, 30425588 Bytes
Unpacking '/content/emsdk/downloads/node-v22.16.0-linux-x64.tar.xz' to '/content/emsdk/node/22.16.0_64bit'
Done installing tool 'node-22.16.0-64bit'.
Installing tool 'releases-

## Build

In [ ]:
%%shell

pushd ./emsdk
source ./emsdk_env.sh
popd

git clone https://github.com/k2-fsa/sherpa-onnx
cd sherpa-onnx

cd wasm/kws/assets

wget -q https://github.com/k2-fsa/sherpa-onnx/releases/download/kws-models/sherpa-onnx-kws-zipformer-gigaspeech-3.3M-2024-01-01.tar.bz2
tar xvf sherpa-onnx-kws-zipformer-gigaspeech-3.3M-2024-01-01.tar.bz2
rm sherpa-onnx-kws-zipformer-gigaspeech-3.3M-2024-01-01.tar.bz2

# Note it is not an error that we rename encoder.int8.onnx to encoder.onnx

mv sherpa-onnx-kws-zipformer-gigaspeech-3.3M-2024-01-01/encoder-epoch-12-avg-2-chunk-16-left-64.onnx encoder.onnx
mv sherpa-onnx-kws-zipformer-gigaspeech-3.3M-2024-01-01/decoder-epoch-12-avg-2-chunk-16-left-64.onnx decoder.onnx
mv sherpa-onnx-kws-zipformer-gigaspeech-3.3M-2024-01-01/joiner-epoch-12-avg-2-chunk-16-left-64.onnx joiner.onnx
mv sherpa-onnx-kws-zipformer-gigaspeech-3.3M-2024-01-01/tokens.txt ./
rm -rf sherpa-onnx-kws-zipformer-gigaspeech-3.3M-2024-01-01/

cd ../../..

./build-wasm-simd-kws.sh

Se truncaron las últimas líneas 5000 del resultado de transmisión.
  698 | #      define _LIBCPP_DEPRECATED __attribute__((__deprecated__))
      |                                                 ^
In file included from /content/sherpa-onnx/build-wasm-simd-asr/_deps/simple-sentencepiece-src/ssentencepiece/csrc/ssentencepiece.cc:19:
In file included from /content/sherpa-onnx/build-wasm-simd-asr/_deps/simple-sentencepiece-src/ssentencepiece/csrc/ssentencepiece.h:24:
/content/sherpa-onnx/build-wasm-simd-asr/_deps/simple-sentencepiece-src/ssentencepiece/csrc/threadpool.h:86:37: warning: 'result_of<(lambda at /content/sherpa-onnx/build-wasm-simd-asr/_deps/simple-sentencepiece-src/ssentencepiece/csrc/ssentencepiece.cc:204:41) ()>' is deprecated [-Wdeprecated-declarations]
   86 |   using return_type = typename std::result_of<F(Args...)>::type;
      |                                     ^
/content/sherpa-onnx/build-wasm-simd-asr/_deps/simple-sentencepiece-src/ssentencepiece/csrc/ssentencepie

In [ ]:
%%shell

ls -lh
echo "---"
ls -lh sherpa-onnx/build-wasm-simd-kws/install/bin/wasm/kws

total 12K
drwxr-xr-x 12 root root 4.0K Aug  8 05:09 emsdk
drwxr-xr-x  1 root root 4.0K Aug  6 13:39 sample_data
drwxr-xr-x 32 root root 4.0K Aug  8 05:09 sherpa-onnx
---
total 25M
-rw-r--r-- 1 root root  11K Aug  8 05:09 app-asr.js
-rw-r--r-- 1 root root 3.7K Aug  8 05:09 index.html
-rw-r--r-- 1 root root  38K Aug  8 05:09 sherpa-onnx-asr.js
-rw-r--r-- 1 root root  14M Aug  8 05:19 sherpa-onnx-wasm-main-asr.data
-rw-r--r-- 1 root root  79K Aug  8 05:26 sherpa-onnx-wasm-main-asr.js
-rw-r--r-- 1 root root  11M Aug  8 05:26 sherpa-onnx-wasm-main-asr.wasm


## Download the generated files to your local computer

In [ ]:
%%shell

mkdir -p sherpa-onnx-wasm-kws
cp -v sherpa-onnx/build-wasm-simd-kws/install/bin/wasm/kws/* ./sherpa-onnx-wasm-kws

tar cjvf ./sherpa-onnx-wasm-kws.tar.bz2 ./sherpa-onnx-wasm-kws


'sherpa-onnx/build-wasm-simd-asr/install/bin/wasm/asr/app-asr.js' -> './sherpa-onnx-wasm-asr/app-asr.js'
'sherpa-onnx/build-wasm-simd-asr/install/bin/wasm/asr/index.html' -> './sherpa-onnx-wasm-asr/index.html'
'sherpa-onnx/build-wasm-simd-asr/install/bin/wasm/asr/sherpa-onnx-asr.js' -> './sherpa-onnx-wasm-asr/sherpa-onnx-asr.js'
'sherpa-onnx/build-wasm-simd-asr/install/bin/wasm/asr/sherpa-onnx-wasm-main-asr.data' -> './sherpa-onnx-wasm-asr/sherpa-onnx-wasm-main-asr.data'
'sherpa-onnx/build-wasm-simd-asr/install/bin/wasm/asr/sherpa-onnx-wasm-main-asr.js' -> './sherpa-onnx-wasm-asr/sherpa-onnx-wasm-main-asr.js'
'sherpa-onnx/build-wasm-simd-asr/install/bin/wasm/asr/sherpa-onnx-wasm-main-asr.wasm' -> './sherpa-onnx-wasm-asr/sherpa-onnx-wasm-main-asr.wasm'
./sherpa-onnx-wasm-asr/
./sherpa-onnx-wasm-asr/sherpa-onnx-wasm-main-asr.js
./sherpa-onnx-wasm-asr/sherpa-onnx-wasm-main-asr.wasm
./sherpa-onnx-wasm-asr/sherpa-onnx-wasm-main-asr.data
./sherpa-onnx-wasm-asr/index.html
./sherpa-onnx-wasm-a

In [ ]:
%%shell
ls -lh

total 15M
drwxr-xr-x 12 root root 4.0K Aug  8 05:09 emsdk
drwxr-xr-x  1 root root 4.0K Aug  6 13:39 sample_data
drwxr-xr-x 32 root root 4.0K Aug  8 05:09 sherpa-onnx
drwxr-xr-x  2 root root 4.0K Aug  8 05:26 sherpa-onnx-wasm-asr
-rw-r--r--  1 root root  15M Aug  8 05:26 sherpa-onnx-wasm-asr.tar.bz2


In [ ]:
from google.colab import files
files.download('sherpa-onnx-wasm-kws.tar.bz2')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Please follow
https://k2-fsa.github.io/sherpa/onnx/wasm/build.html
for how to use the downloaded files.